In [1]:
import tweepy
#import cnfg     #doesn't work for me.  
from tqdm import tqdm

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:

auth = tweepy.OAuthHandler("#########################",
                           "##################################################")
auth.set_access_token("##################################################",
                      "#############################################")

api=tweepy.API(auth)

In [3]:
max_tweets=1

#Tweepy Cursor handles pagination .. 

for tweet in tweepy.Cursor(api.search,q="@AmericanAir").items(max_tweets):
    print(tweet)

Status(_api=<tweepy.api.API object at 0x0000027C33A5D898>, _json={'created_at': 'Fri Nov 23 00:55:44 +0000 2018', 'id': 1065770843182153729, 'id_str': '1065770843182153729', 'text': '@AmericanAir 2nd time ever flying w American on the tail. Hardcore JetBlue flyer. Dropped 25k points from a Citiban… https://t.co/Opu2Vy4Ra2', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'AmericanAir', 'name': 'American Airlines', 'id': 22536055, 'id_str': '22536055', 'indices': [0, 12]}], 'urls': [{'url': 'https://t.co/Opu2Vy4Ra2', 'expanded_url': 'https://twitter.com/i/web/status/1065770843182153729', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Lite</a>', 'in_reply_to_status_id': 1065769463545544704, 'in_reply_to_status_id_str': '1065769463545544704', 'in_reply_to_user_id': 22536055, 'in_re

In [ ]:
results=[]


for tweet in tweepy.Cursor(api.search,q="@AmericanAir",wait_on_rate_limit=True).items(100000):
    results.append(tweet)

In [ ]:
#  import pandas as pd
def structure_results(results):
    id_list=[tweet.id for tweet in results]
    data=pd.DataFrame(id_list,columns=['id'])
    
    data["text"]= [tweet.text.encode('utf-8') for tweet in results]
    data["datetime"]=[tweet.created_at for tweet in results]
    data["Location"]=[tweet.place for tweet in results]
    
    return data

In [ ]:
pd.set_option('max_colwidth',180)
data=structure_results(results)
data

In [ ]:
from pymongo import MongoClient
from tweepy import Stream
from tweepy.streaming import StreamListener

import json

In [ ]:
# Connect to monogodb server.  Make sure it's running first!
client = MongoClient('localhost', 27017)

In [ ]:
client.list_database_names()

In [ ]:
# Connect to the database for the first time
twitter_db = client['twitter_database']

# Create a collection
tweet_collection = twitter_db.tweets

In [ ]:
tweet_dict_list[0]

In [ ]:
tweet_collection.insert_one(tweet_dict_list[0])

In [ ]:
tweet_collection.find_one()

In [ ]:
tweet_collection.insert_many(tweet_dict_list[1:5])

In [ ]:
for t in tweet_collection.find():
    print(t)

In [ ]:
for t in tweet_collection.find({'favorite_count': {'$gt':0}}):
    print(t)

In [ ]:
client.drop_database('twitter_database')

In [ ]:
class listener(StreamListener):

    def on_data(self, data):
        data = json.loads(data)
        
        # Pull the fields we want, and throw it into our mongodb database
        if data['lang'] == 'en':
            tweet_document = {}

            tweet_document['created_at'] = data['created_at']
            tweet_document['favorite_count'] = data['favorite_count']
            tweet_document['retweet_count'] = data['retweet_count']

            #Get full text if it is an extended tweet
            if 'extended_tweet' in data.keys():
                tweet_document['text'] = data['extended_tweet']['full_text']
            else:
                tweet_document['text'] = data['text']

            tweet_document['screen_name'] = data['user']['screen_name']

            text = TextBlob(tweet_document['text']).sentiment

            tweet_document['sentiment'] = text.polarity
            tweet_document['subjectivity'] = text.subjectivity

           # print(tweet_document)
           # print('\n')
            tweet_collection.insert_one(tweet_document)
        
        return True

    def on_error(self, status):
        print(status)

In [ ]:
twitterStream = Stream(auth, listener())
data = twitterStream.filter(track=["bitcoin"])

In [ ]:
twitter_db['tweets'].find_one()

In [ ]:
res = twitter_db['tweets'].find().sort([('created_at',-1)]).limit(100)

In [ ]:
res_list = [item for item in res]

In [ ]:
df = pd.DataFrame(res_list)
df.head()